In [17]:
import pyodbc
import pandas as pd
import glob
import sqlite3

In [ ]:
# Connecting to the SSMS from notebook

server_name = 'ServerName'  # Replace with your server name or 'localhost' for a local database
database_name = 'YourDatabaseName'  # Replace with your database name
username = 'YourUsername'  # Replace with your SQL Server username
password = 'YourPassword'  # Replace with your SQL Server password

connection_string = f'DRIVER={{SQL Server}};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password}'
connection = pyodbc.connect(connection_string)


## Read multiple csv files

In [29]:
# file pattern and directory path
file_pattern = '*.csv' 
directory_path = 'data'

# Using glob.glob to get a list of matching file names
file_names = glob.glob('data/*.csv')

# Print the list of file names
csv_files = []
table_name = []
for file_name in file_names:
    csv_files.append(file_name)
    file_name = str(file_name.split('.')[0].split('\\')[-1])
    table_name.append(file_name)
    print(file_name)


accounts
orders
region
sales_reps
web_events


In [30]:
csv_files

['data\\accounts.csv',
 'data\\orders.csv',
 'data\\region.csv',
 'data\\sales_reps.csv',
 'data\\web_events.csv']

In [31]:
table_name

['accounts', 'orders', 'region', 'sales_reps', 'web_events']

In [18]:
# Connect to or create the SQLite database
conn = sqlite3.connect("parch_and_posey.db")

In [32]:
#csv_files = ["file1.csv", "file2.csv", "file3.csv"]

for file in csv_files:
    table_name = str(file_name.split('.')[0].split('\\')[-1])
    df = pd.read_csv(file)  # Read CSV file into a DataFrame
    df.to_sql(table_name, conn, if_exists="replace", index=False)  # Insert data into SQLite table


In [34]:
%load_ext sql

ModuleNotFoundError: No module named 'sql'

In [33]:
%%sql

SELECT name
FROM sqlite_master
WHERE type IN ('table', 'view')
AND name NOT LIKE 'sqlite_%' ORDER BY 1

UsageError: Cell magic `%%sql` not found.
